<a href="https://colab.research.google.com/github/tatiana-iazykova/project_R/blob/main/R_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

We download the whole dataset from Kaggle. However, we won't need anything but the texts from there. Every article is in its own `.json` document. So we combine them into one `.csv` file per category so that our work will be easier 

In [ ]:
%%capture
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/HSE/Thesis"

In [ ]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

In [ ]:
!unzip /content/drive/MyDrive/HSE/Thesis/CORD-19-research-challenge.zip -d train

unzip:  cannot find or open /content/drive/MyDrive/HSE/Thesis/CORD-19-research-challenge.zip, /content/drive/MyDrive/HSE/Thesis/CORD-19-research-challenge.zip.zip or /content/drive/MyDrive/HSE/Thesis/CORD-19-research-challenge.zip.ZIP.


In [ ]:
%cd /content/train/document_parses/pmc_json

/content/train/document_parses/pmc_json


In [ ]:
def get_json(dir):
  df = pd.DataFrame(columns=['text', 'article_id', "title"])
  for i in tqdm(os.listdir(dir)):
    if i.endswith(".json"):
      with open(i) as f:
        data = json.load(f)
      text = ''
      for l in data['body_text']:
        text += l['text'] + ' '
      d = {'text': text, 'article_id': data["paper_id"], 'title': data["metadata"]['title']}
      df = df.append(d, ignore_index=True)
  return df

In [ ]:
df_1 = get_json('.')

In [ ]:
df_1.to_csv('/content/pmc_json_df.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/HSE/Thesis/pmc_json_df.csv')

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.to_csv('/content/pmc_json_df.csv', index=False)

In [ ]:
!cp -r /content/pmc_json_df.csv /content/drive/MyDrive/HSE/Thesis

In [ ]:
%cd ..
%cd /pdf_json

In [ ]:
df_2 = get_json('.')

In [ ]:
df_2.to_csv('/content/pdf_json_df.csv', index=False)

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/HSE/Thesis/pdf_json_df.csv')

In [ ]:
df3 = df3.drop('Unnamed: 0', axis=1)

In [ ]:
df3.to_csv('/content/pdf_json_df.csv', index=False)

In [ ]:
!cp -r /content/pdf_json_df.csv /content/drive/MyDrive/HSE/Thesis

# Actual work

This is an example for the `pmc_json_df.csv` file. The process for `pdf_json_df.csv` is the same.

In [ ]:
df = pd.read_csv('pmc_json_df.csv')

In [ ]:
df.article_id = df.article_id.astype("category")
df.title = df.title.fillna('None')

In [ ]:
covid_study = df[df.text.str.contains('corona( |-)?virus|covid-19|SARS-CoV|2019-nCoV|MERS-CoV', case=False)]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
vaccines = covid_study[covid_study.text.str.contains('vaccin(e|ation)|Pfizer|AstraZeneca|Sputnik( V)?|BioNTech|Sinopharm|Sinovac|Moderna|Covaxin', case=False)]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
len(vaccines)

35927

In [ ]:
tmp = vaccines[vaccines.text.str.contains('(and so on)|(shouldn\'t)|(aren\'t)|(isn\'t)|(can\'t)|(ha(ve|s)n\'t)|(won\'t)|([^A-z](g|w)onna[^A-z])|([^A-z]gimme[^A-z])|(kung flu)|((China|Asian) virus)|([^A-z]Rona[^A-z])|([^A-z]corona(reezy)?[^A-z-]^(virus))|([^A-z]Coro(ne)?s[^A-z])|(Boomer Remover)|(Senior Deleter)|(Geriatric Dispatcher)|(covid(dy|sky|z))', case=False)]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
len(tmp)

2190

In [ ]:
import re
def cute_regex(x):
  expr = ['and so on', 'aren\'t','can\'t','haven\'t', 'hasn\'t', 'won\'t', '[^A-z]gonna[^A-z]', '[^A-z]wonna[^A-z]',
          '[^A-z]gimme[^A-z]', 'kung flu', 'China virus', 'Asian virus', 'shouldn\'t', 'isn\'t',
          '[^A-z]Rona[^A-z]', 'Coronareezy', '[^A-z]corona[^A-z-]',  '[^A-z]Corones[^A-z]', '[^A-z]coros[^A-z]', 'Boomer Remover',
          'Senior Deleter', 'Geriatric Dispatcher', 'coviddy', 'covidsky', 'convidz']
  res = []
  for e in expr:
    res.append(re.findall(e, x, flags=re.IGNORECASE))
  return [item for sublist in res for item in sublist if item]

In [ ]:
tmp['contracted'] = tmp.text.apply(cute_regex)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def count_apply(x,y):
  f = y.split()
  return len(x)/len(f)

In [ ]:
tmp['count_contr'] = tmp.apply(lambda x: count_apply(x['contracted'], x['text']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
tmp = tmp.drop(['article_id', 'title'], axis=1)

In [ ]:
tmp.count_contr.max()

0.012029912214154113

In [ ]:
tmp.to_csv('dataset_science_pt1.csv', index=False)